<a href="https://colab.research.google.com/github/CamCranda11/MLFA25Project/blob/main/MLProjectModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

song_data = pd.read_csv('spotify_data.csv')

print("--- Original Data ---")
print(song_data.head())
print("\n")

--- Original Data ---
   Unnamed: 0    artist_name        track_name                track_id  \
0           0     Jason Mraz   I Won't Give Up  53QF56cjZA9RTuuMZDrSA6   
1           1     Jason Mraz  93 Million Miles  1s8tP3jP4GZcyHDsjvw218   
2           2  Joshua Hyslop  Do Not Let Me Go  7BRCa8MPiyuvr2VU3O9W0F   
3           3   Boyce Avenue          Fast Car  63wsZUhUZLlh1OsyrZq7sz   
4           4   Andrew Belle  Sky's Still Blue  6nXIYClvJAfi6ujLiKqEq8   

   popularity  year     genre  danceability  energy  key  loudness  mode  \
0          68  2012  acoustic         0.483   0.303    4   -10.058     1   
1          50  2012  acoustic         0.572   0.454    3   -10.286     1   
2          57  2012  acoustic         0.409   0.234    3   -13.711     1   
3          58  2012  acoustic         0.392   0.251   10    -9.845     1   
4          54  2012  acoustic         0.430   0.791    6    -5.419     0   

   speechiness  acousticness  instrumentalness  liveness  valence    tempo  

In [3]:
features_to_cluster = ['danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence']
features_df = song_data[features_to_cluster].copy()

mode_value = features_df['mode'].mode()[0]
features_df['mode'] = features_df['mode'].fillna(mode_value)

scaler = StandardScaler()
scaled_features_df = scaler.fit_transform(features_df)

print("--- Scaled Features (First 5 Rows) ---")
print(scaled_features_df[:5])
print("\n")

--- Scaled Features (First 5 Rows) ---
[[-0.29509342 -1.24461718  0.75872495 -0.39352278  1.04922991 -0.69122871
  -1.17892497]
 [ 0.18734904 -0.68639325  0.75872495 -0.52833738  0.43794003 -0.69119118
   0.22134908]
 [-0.69622536 -1.49969964  0.75872495 -0.47709206  0.04637646 -0.69109175
  -1.15658017]
 [-0.78837729 -1.43685324  0.75872495 -0.44555648  1.36755137 -0.69122871
   0.19528015]
 [-0.58239062  0.55944426 -1.31800069 -0.49364824 -0.70125643 -0.63836256
  -0.88844259]]




In [4]:
optimal_k = 50

kmeans_model = KMeans(n_clusters=optimal_k, init='k-means++', n_init=10, random_state=42)
kmeans_model.fit(scaled_features_df)

cluster_labels = kmeans_model.labels_

song_cluster_df = song_data.copy()
song_cluster_df['cluster_id'] = cluster_labels

print("--- Data with Cluster IDs ---")
display(song_cluster_df.head())
print("\n")

--- Data with Cluster IDs ---


Unnamed: 0    artist_name        track_name                track_id  \
0           0     Jason Mraz   I Won't Give Up  53QF56cjZA9RTuuMZDrSA6   
1           1     Jason Mraz  93 Million Miles  1s8tP3jP4GZcyHDsjvw218   
2           2  Joshua Hyslop  Do Not Let Me Go  7BRCa8MPiyuvr2VU3O9W0F   
3           3   Boyce Avenue          Fast Car  63wsZUhUZLlh1OsyrZq7sz   
4           4   Andrew Belle  Sky's Still Blue  6nXIYClvJAfi6ujLiKqEq8   

   popularity  year     genre  danceability  energy  key  ...  mode  \
0          68  2012  acoustic         0.483   0.303    4  ...     1   
1          50  2012  acoustic         0.572   0.454    3  ...     1   
2          57  2012  acoustic         0.409   0.234    3  ...     1   
3          58  2012  acoustic         0.392   0.251   10  ...     1   
4          54  2012  acoustic         0.430   0.791    6  ...     0   

   speechiness  acousticness  instrumentalness  liveness  valence    tempo  \
0       0.0429        0.6940          0.000000    0.1150    0.139  133.406   
1       0.0258        0.4770          0.000014    0.0974    0.515  140.182   
2       0.0323        0.3380          0.000050    0.0895    0.145  139.832   
3       0.0363        0.8070          0.000000    0.0797    0.508  204.961   
4       0.0302        0.0726          0.019300    0.1100    0.217  171.864   

   duration_ms  time_signature  cluster_id  
0       240166               3          17  
1       216387               4          27  
2       158960               4          17  
3       304293               4           1  
4       244320               4          40  

[5 rows x 21 columns]

In [5]:
def recommend_songs(input_song_name, input_artist_name, data_df, num_recs=3):
    """
    Recommends songs from the same cluster and genre as the input song and artist.
    Includes track_id for generating Spotify links.
    """
    try:
        song_row = data_df[
            (data_df['track_name'] == input_song_name) &
            (data_df['artist_name'] == input_artist_name)
        ].iloc[0]
        song_cluster = song_row['cluster_id']
        song_genre = song_row['genre']

        recommendations = data_df[
            (data_df['cluster_id'] == song_cluster) &
            (data_df['genre'] == song_genre) &
            ((data_df['track_name'] != input_song_name) | (data_df['artist_name'] != input_artist_name))
        ]

        if len(recommendations) == 0:
            return "No similar songs found in the same genre."

        return recommendations.sample(min(num_recs, len(recommendations)))

    except IndexError:
        return f"Song '{input_song_name}' by {input_artist_name} not found in the dataset."

In [34]:
my_song = 'Tongues'
my_artist = 'Joywave'
recommendations = recommend_songs(my_song, my_artist, song_cluster_df, num_recs=5)

try:
    input_song_row = song_cluster_df[
        (song_cluster_df['track_name'] == my_song) &
        (song_cluster_df['artist_name'] == my_artist)
    ].iloc[0]
    input_song_track_id = input_song_row['track_id']
    print(f"--- Input Song: '{my_song}' by {my_artist} https://open.spotify.com/track/{input_song_track_id} ---")
    display(input_song_row[['genre', 'danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'cluster_id']])

    print(f"\n--- Recommendations based on '{my_song}' by {my_artist} ---")
    if isinstance(recommendations, pd.DataFrame):
        display(recommendations[['track_name', 'artist_name', 'track_id', 'genre', 'danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence', 'cluster_id']])
    else:
        print(recommendations)


except IndexError:
    print(f"Song '{my_song}' by {my_artist} not found in the dataset.")

--- Input Song: 'Tongues' by Joywave https://open.spotify.com/track/5mCprFWOqe0jB96v9RhLBo ---


genre               alt-rock
danceability           0.827
energy                 0.682
mode                       0
speechiness           0.0435
acousticness         0.00048
instrumentalness      0.0187
valence                0.933
cluster_id                 2
Name: 162989, dtype: object


--- Recommendations based on 'Tongues' by Joywave ---


track_name      artist_name                track_id     genre  \
928095             Hablan      Dread Mar I  3aazT0KqFmg8kkJCmXqDJV  alt-rock   
667860           El Viejo   La Vela Puerca  7gKqjNMM4ikt7KFTRsgZgL  alt-rock   
367931               Chau  No Te Va Gustar  3uP1USIwwDpOA8uw0LjY2P  alt-rock   
585090   Que Va A Suceder           La Ley  2lSO0InO0iMn3KLa4wDtOr  alt-rock   
163254  Make Your Mind Up        Moon Taxi  4EAoAeIwUUvevkPLvhwTpx  alt-rock   

        danceability  energy  mode  speechiness  acousticness  \
928095         0.773   0.412     0       0.0468        0.2310   
667860         0.699   0.744     0       0.0275        0.0696   
367931         0.807   0.377     0       0.0520        0.1510   
585090         0.687   0.406     0       0.0337        0.1600   
163254         0.746   0.839     0       0.0298        0.0046   

        instrumentalness  valence  cluster_id  
928095          0.000003    0.947           2  
667860          0.000000    0.810           2  
367931          0.001650    0.917           2  
585090          0.055300    0.923           2  
163254          0.000093    0.757           2

In [26]:
import pandas as pd

artist_name = "Joywave"

pd.set_option('display.max_rows', None)

artist_songs = song_data[song_data['artist_name'] == artist_name]

if len(artist_songs) > 0:
    print(f"--- Songs by {artist_name} ---")
    display(artist_songs)
else:
    print(f"No songs found for artist: {artist_name}")

pd.reset_option('display.max_rows')

--- Songs by Joywave ---


Unnamed: 0 artist_name                             track_name  \
2654          2654     Joywave                           Golden State   
34104        34104     Joywave                              True Grit   
34115        34115     Joywave                                Anemone   
162971      162971     Joywave                            Destruction   
162989      162989     Joywave                                Tongues   
163151      163151     Joywave                           Somebody New   
163220      163220     Joywave        Traveling at the Speed of Light   
163239      163239     Joywave                             Nice House   
163358      163358     Joywave                                    Now   
163542      163542     Joywave                             Bad Dreams   
163648      163648     Joywave                                 Parade   
163791      163791     Joywave                               Carry Me   
215417      215417     Joywave                            Incredible?   
215460      215460     Joywave  Life in a Bubble I Blew - bonus track   
254852      254852     Joywave                           It's A Trip!   
255411      255411     Joywave                                  Doubt   
255628      255628     Joywave                               Shutdown   
255655      255655     Joywave              Let's Talk About Feelings   
255657      255657     Joywave                                Content   
255669      255669     Joywave                                 Rumors   
311588      311588     Joywave                             Blastoffff   
423644      423644     Joywave                              Obsession   
423651      423651     Joywave                               F.E.A.R.   
423805      423805     Joywave                           Coming Apart   
423864      423864     Joywave                          Half Your Age   
424336      424336     Joywave             Coming Apart - PVRIS Remix   
479062      479062     Joywave               Every Window Is A Mirror   
532654      532654     Joywave                           After Coffee   
532749      532749     Joywave                           Buy American   

                      track_id  popularity  year      genre  danceability  \
2654    6b6KGKBboxwTgTxtYZYE7I          25  2012   alt-rock         0.483   
34104   1H8EZ7uCaKBEX97B7KoHTI          24  2012  indie-pop         0.517   
34115   0geaQiAZN8AhkxLfyAiUXe          24  2012  indie-pop         0.608   
162971  2iLxXSM7AOzB4RCNzk4bjd          53  2015   alt-rock         0.660   
162989  5mCprFWOqe0jB96v9RhLBo          52  2015   alt-rock         0.827   
163151  2vcfImyMOQjYhnKOVkjoIR          42  2015   alt-rock         0.456   
163220  5zZOhHw2p7VLPau3A29hJw          39  2015   alt-rock         0.570   
163239  0GAvMukpDYssdZgMLsSuhx          38  2015   alt-rock         0.549   
163358  2UbzSDEYP8eCg5K2a8d3PM          35  2015   alt-rock         0.575   
163542  0B7B8DAnYcWHqcwqOpenth          32  2015   alt-rock         0.733   
163648  2Ce6HBGVdo8ky2kjN5ywNR          31  2015   alt-rock         0.587   
163791  6jtNwYVOfaTgiZvsPsm8lG          29  2015   alt-rock         0.546   
215417  66voD4VvfETwDRgnhoIypu          32  2016   alt-rock         0.638   
215460  2gcqVTesDy08DOB2rV2Odt          31  2016   alt-rock         0.655   
254852  6IY3cC281mz9q0bViUEldz          58  2017   alt-rock         0.673   
255411  62a3F9ewBKl9qOxL4KJfju          35  2017   alt-rock         0.525   
255628  7zssIYCjv1ouphwqvD5Dao          32  2017   alt-rock         0.531   
255655  3dq922jmpJ9eMl5PChgvxy          31  2017   alt-rock         0.558   
255657  6N3zpuSStSBPTKiDvlecS5          33  2017   alt-rock         0.497   
255669  4ZomBOkbVdvehlFsPCdEkH          32  2017   alt-rock         0.725   
311588  4fXcjHURFEtoncSd03fJfe          39  2018   alt-rock         0.573   
423644  3BUEXULG9L4AZDEh9CePFE          48  2020   alt-rock         0.656   
423651  4YXkB2Ti8KBX8dgUnhLoNg          48  20